Do an link  prediction of graph Assignment


In [1]:
!pip install torch_geometric

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 467 kB 6.5 MB/s 
  Created wheel for torch-geometric: filename=torch_geometric-2.1.0.post1-py3-none-any.whl size=689859 sha256=912c184fd9a1a5e906b1be05aea9a776c3975e6784a8810c67fbc24abbe879ac
  Stored in directory: /root/.cache/pip/wheels/d1/cb/43/f7f2e472de4d7cff31bceddadc36d634e1e545fbc17961c282
Successfully built torch-geometric


In [2]:
!pip install torch_sparse

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.1 MB 6.6 MB/s 
  Created wheel for torch-sparse: filename=torch_sparse-0.6.15-cp37-cp37m-linux_x86_64.whl size=1577340 sha256=6786cf6eb71f4921cb1b9f6499d4a5340028ad364f232aca5f48d2b0280735c7
  Stored in directory: /root/.cache/pip/wheels/15/68/4d/1414be5c2c622bad35364e13213180797717b6d4b8923936dc
Successfully built torch-sparse


In [3]:
!pip install torch_scatter

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for torch-scatter: filename=torch_scatter-2.0.9-cp37-cp37m-linux_x86_64.whl size=2657113 sha256=bbf792882eb095f53261781f98150e1f8c1ac55cb15165370a4f1a5fe2aa5ba6
  Stored in directory: /root/.cache/pip/wheels/dd/57/a3/42ea193b77378ce634eb9454c9bc1e3163f3b482a35cdee4d1
Successfully built torch-scatter


In [4]:
import os.path as osp

import torch
from sklearn.metrics import roc_auc_score

import torch_geometric.transforms as T
from torch_geometric.datasets import Planetoid
from torch_geometric.nn import GCNConv
from torch_geometric.utils import negative_sampling

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [6]:
transform = T.Compose([
    T.NormalizeFeatures(),
    T.ToDevice(device),
    T.RandomLinkSplit(num_val=0.05, num_test=0.1, is_undirected=True,
                      add_negative_train_samples=False),
])

In [7]:
dataset = 'Cora'
path = osp.join('.', 'data', dataset)
#path = osp.join(osp.dirname(osp.realpath(__file__)), '..', 'data', 'Planetoid')
dataset = Planetoid(path, name='Cora', transform=transform)

Processing...
Done!


In [8]:

train_data, val_data, test_data = dataset[0]

In [9]:
class Net(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super().__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)

    def encode(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        return self.conv2(x, edge_index)

    def decode(self, z, edge_label_index):
        return (z[edge_label_index[0]] * z[edge_label_index[1]]).sum(dim=-1)

    def decode_all(self, z):
        prob_adj = z @ z.t()
        return (prob_adj > 0).nonzero(as_tuple=False).t()

In [10]:
model = Net(dataset.num_features, 128, 64).to(device)
optimizer = torch.optim.Adam(params=model.parameters(), lr=0.01)
criterion = torch.nn.BCEWithLogitsLoss()

In [11]:
def train():
    model.train()
    optimizer.zero_grad()
    z = model.encode(train_data.x, train_data.edge_index)

    # We perform a new round of negative sampling for every training epoch:
    neg_edge_index = negative_sampling(
        edge_index=train_data.edge_index, num_nodes=train_data.num_nodes,
        num_neg_samples=train_data.edge_label_index.size(1), method='sparse')

    edge_label_index = torch.cat(
        [train_data.edge_label_index, neg_edge_index],
        dim=-1,
    )
    edge_label = torch.cat([
        train_data.edge_label,
        train_data.edge_label.new_zeros(neg_edge_index.size(1))
    ], dim=0)

    out = model.decode(z, edge_label_index).view(-1)
    loss = criterion(out, edge_label)
    loss.backward()
    optimizer.step()
    return loss

In [12]:
@torch.no_grad()
def test(data):
    model.eval()
    z = model.encode(data.x, data.edge_index)
    out = model.decode(z, data.edge_label_index).view(-1).sigmoid()
    return roc_auc_score(data.edge_label.cpu().numpy(), out.cpu().numpy())


best_val_auc = final_test_auc = 0
for epoch in range(1, 101):
    loss = train()
    val_auc = test(val_data)
    test_auc = test(test_data)
    if val_auc > best_val_auc:
        best_val_auc = val_auc
        final_test_auc = test_auc
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, Val: {val_auc:.4f}, '
          f'Test: {test_auc:.4f}')

Epoch: 001, Loss: 0.6930, Val: 0.7178, Test: 0.6720
Epoch: 002, Loss: 0.6805, Val: 0.7082, Test: 0.6621
Epoch: 003, Loss: 0.7216, Val: 0.7014, Test: 0.6653
Epoch: 004, Loss: 0.6756, Val: 0.6917, Test: 0.6762
Epoch: 005, Loss: 0.6849, Val: 0.6946, Test: 0.6922
Epoch: 006, Loss: 0.6895, Val: 0.6947, Test: 0.6983
Epoch: 007, Loss: 0.6910, Val: 0.6895, Test: 0.6945
Epoch: 008, Loss: 0.6912, Val: 0.6845, Test: 0.6856
Epoch: 009, Loss: 0.6907, Val: 0.6801, Test: 0.6795
Epoch: 010, Loss: 0.6894, Val: 0.6763, Test: 0.6749
Epoch: 011, Loss: 0.6871, Val: 0.6704, Test: 0.6688
Epoch: 012, Loss: 0.6833, Val: 0.6658, Test: 0.6631
Epoch: 013, Loss: 0.6781, Val: 0.6630, Test: 0.6584
Epoch: 014, Loss: 0.6751, Val: 0.6576, Test: 0.6543
Epoch: 015, Loss: 0.6758, Val: 0.6547, Test: 0.6530
Epoch: 016, Loss: 0.6715, Val: 0.6555, Test: 0.6579
Epoch: 017, Loss: 0.6637, Val: 0.6677, Test: 0.6741
Epoch: 018, Loss: 0.6586, Val: 0.6823, Test: 0.6965
Epoch: 019, Loss: 0.6553, Val: 0.6910, Test: 0.7070
Epoch: 020, 

In [13]:
print(f'Final Test: {final_test_auc:.4f}')

z = model.encode(test_data.x, test_data.edge_index)
final_edge_index = model.decode_all(z)

Final Test: 0.8828


References



*   http://cs230.stanford.edu/projects_spring_2020/reports/38854344.pdf




*   https://colab.research.google.com/drive/1f2KVSVsk5f4dpZlxNE0RBIHowqDs3KSp#scrollTo=dQ98qUOAt5JK



*  https://colab.sandbox.google.com/github/AntonioLonga/PytorchGeometricTutorial/blob/main/Tutorial12/Tutorial12%20GAE%20for%20link%20prediction.ipynb







